# Setup

## Load Modules

In [ ]:
%matplotlib notebook

# Standard library
import os.path, logging, collections

# External modules
from neuron import h
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import bluepyopt.ephys as ephys

# Our modules
from bgcellmodels.common import electrotonic, treeutils, analysis, logutils
from bgcellmodels.morphology import morph_ni, morph_gui

from bgcellmodels.models.STN import GilliesWillshaw as gillies
from bgcellmodels.models.STN import Miocinovic2006 as miocinovic
from bgcellmodels.models.axon.mcintyre2002 import AxonMcintyre2002

## Settings

In [ ]:
gillies.load_mechanisms()
default_template = 'STN_morph_arcdist'

class CellModel(object):
    """ Wrapper class to store NEURON cell data. """
    pass

# Global variables
nrnsim = ephys.simulators.NrnSimulator(dt=0.025, cvode_active=False)

logger = logging.getLogger('STN_MORPH')
logger.setLevel(logging.DEBUG)
logging.basicConfig(format=logutils.DEFAULT_FORMAT)

In [ ]:
# Plotting options
page_width = 5.0
fig_aspect_ratio = 2.2 # 8.0 / 3.0
fig_width = 0.8 * page_width
fig_height = fig_width / fig_aspect_ratio

# Style of figures (default colors etc.): see https://matplotlib.org/gallery/style_sheets/style_sheets_reference.html
plt.style.use('default')

# fix bug for notebook backend where figures not shown
if matplotlib.get_backend() == 'nbAgg':
    from bgcellmodels.extensions.jupyter import jupyterutil
    jupyterutil.notebook_show_figs_after_exception()

## Functions

In [ ]:
def template_with_morphology(template_name,
                             morphology_path,
                             streamlines_path,
                             streamline_index=0,
                             **kwargs):
    """
    Instantiate template with given morphology and axon
    
    @param    **kwargs
              All remaining keyword arguments are set as attributes
              on the instantiated template.
    """
    miocinovic.load_template(template_name) # xopen -> loads once
    template_constructor = getattr(h, template_name)
    
    # Instantiate template
    icell = template_constructor()
    icell.with_extracellular = 0
    for k,v in kwargs.items():
        setattr(icell, k, v)
    
    wrapper = CellModel()
    wrapper.icell = icell

    # Load morphology into template
    morphology = ephys.morphologies.NrnFileMorphology(morphology_path, do_replace_axon=False)
    morphology.instantiate(sim=nrnsim, icell=icell)

    # Setup biophysical properties
    icell.del_unused_sections()
    icell.insert_biophys()
    nseg_extra = electrotonic.set_min_nseg_hines(icell.all, f_lambda=100.0)
    print("Created {} extra segments to satisfy Hines' rule".format(nseg_extra))
    icell.set_biophys_spatial()

    if streamlines_path is not None:
        # Load axon coordinates
        axon_builder = AxonMcintyre2002(logger=logger)
        tracks_coords = morph_ni.load_streamlines(streamlines_path, max_num=1, min_length=2.0)
        axon_coords = tracks_coords[0]

        # Build axon
        axon_initial_secs = list(icell.axonal)
        axon_terminal_secs = treeutils.leaf_sections(axon_initial_secs[0], subtree=True)
        assert len(axon_terminal_secs) == 1
        axon_end_sec = axon_terminal_secs[0]

        axon = axon_builder.build_along_streamline(axon_coords,
                    terminate='nodal_cutoff', interp_method='cartesian',
                    parent_cell=icell, parent_sec=axon_end_sec,
                    connection_method='translate_axon_start')
    
        wrapper.axon = axon

    return wrapper


def template_predefined_morphology(template_name='STN_morph_type1RD', with_extracellular=False):
    """
    Load template with built-in morphology.
    """
    miocinovic.load_template(template_name) # xopen -> loads once
    template_constructor = getattr(h, template_name)
    
    # Instantiate template
    icell = template_constructor(with_extracellular)
    return icell

## Measurement

In [ ]:
electrotonic_measures = {
    'Ztransfer': {
        'function': electrotonic.measure_transfer_impedance,
        'label': 'Transfer impedance',
        'units': 'Ohm',
    },
    'Ai': {
        'function': electrotonic.measure_current_transfer,
        'label': 'Current transfer ratio',
        'units': 'ratio',
    },
    'Av': {
        'function': electrotonic.measure_voltage_transfer,
        'label': 'Voltage transfer ratio',
        'units': 'ratio',
    },
    'Zin': {
        'function': electrotonic.measure_input_impedance,
        'label': 'Input impedance',
        'units': 'Ohm',
    }
}

def plot_electrotonic_structure(root, leaves, measures=None, freq=25.0):
    """
    Plot electrotonic properties along dendritic paths.
    """
    if measures is None:
        plotted_measures = electrotonic_measures.keys()
    else:
        plotted_measures = measures
    measure_funcs = {measure: electrotonic_measures[measure]['function'] for measure in plotted_measures}
        
    # Do measurement
    leaf_path_measurements = electrotonic.measure_along_paths(root, leaves, measure_funcs, freq=freq)

    # Plot measurements
    fig, axes = plt.subplots(len(plotted_measures))

#     measures_ylims = {
#         'Ztransfer': (0, 50),
#         'Ai': (0.5, 1.05),
#         'Av': (0, 1.1),
#         'Zin': (0, 1000.0),
#     }

    for i, measure in enumerate(plotted_measures):
        ax = axes[i]

        # Plot the same measure for each leaf in the same axis
        for j, leaf in enumerate(leaves):
            ax.plot(leaf_path_measurements[j]['pathlen_micron'],
                    leaf_path_measurements[j][measure],
                    label=leaf.name())

            # ax.set_ylim(measures_ylims[measure])
            # ax.legend()
            ax.grid(True)
            ax.set_ylabel('{} ({})'.format(measure, electrotonic_measures[measure]['units']))
            ax.set_title(electrotonic_measures[measure]['label'])
    
    # prevent clipped xlabel
    # fig.subplots_adjust(hspace=0.5)
    # fig.tight_layout()
    return leaf_path_measurements

In [ ]:
import numpy as np

def get_sample(sec, i):
    return np.array([h.x3d(i, sec=sec), h.y3d(i, sec=sec), h.z3d(i, sec=sec), h.diam3d(i, sec=sec)])

## Protocols

In [ ]:
def setup_protocol_SPONT():
    # Gillies (2006) - Fig. 3.B (25 deg, 8Hz), 3.C (35 deg, 12 Hz)
    h.dt = 0.025
    h.tstop = 2000

    h.celsius = 35 # different temp from paper (fig 3B: 25degC, fig. 3C: 35degC)
    h.v_init = -60 # paper simulations use default v_init
    # h.set_aCSF(4) # Set initial ion concentrations from Bevan & Wilson (1999)

    h.init()
    return {}


def setup_protocol_REBOUND(icell):
    # Gillies (2006) - Fig. 3.D, 4 (altered conductances)
    # Original code used -0.25 nA current to hyperpolarize to -75 mV steady state
    clamp = h.SEClamp(icell.soma[0](0.5))
    clamp.dur1 = 0
    clamp.dur2 = 0
    clamp.dur3 = 500
    clamp.amp3 = -75
    
    h.dt = 0.025
    h.tstop = 2000

    h.celsius = 35 # different temp from paper (fig 3B: 25degC, fig. 3C: 35degC)
    h.v_init = -60 # paper simulations use default v_init
    # h.set_aCSF(4) # Set initial ion concentrations from Bevan & Wilson (1999)

    h.init()
    return {'electrodes': [clamp]}


def setup_protocol_PLATEAU(icell):
    # Gillies (2006) - Fig. 10.C
    # Protocol: hyperpolarization to -70 mV using constant current,
    # then apply 0.2 nA, 50 ms depolarizing pulse, return to constant hyperpolarizing current
    soma = icell.soma[0]

    # Current clamp parameters
    amp_hyper = -0.17           # SETPARAM: adjust amplitude for hyperpolarizing to - 70 mV
    amp_pulse = amp_hyper + 0.2 # SETPARAM: adjust depolarizing pulse amplitude based on input resistance

    dur_pulse = 50 
    del_pulse = 1000

    # Make current clamps
    try:
        del clamp
    except NameError:
        pass
    stim1, stim2, stim3 = [h.IClamp(soma(0.5)) for i in range(3)]

    stim1.delay = 0
    stim1.dur = del_pulse
    stim1.amp = amp_hyper

    stim2.delay = del_pulse
    stim2.dur = dur_pulse
    stim2.amp = amp_pulse

    stim3.delay = del_pulse + dur_pulse
    stim3.dur = h.tstop - stim3.delay
    stim3.amp = amp_hyper

    h.dt = 0.025
    h.tstop = 1500

    h.celsius = 30 # different temp from paper (fig 3B: 25degC, fig. 3C: 35degC)
    h.v_init = -60 # paper simulations use default v_init
    # h.set_aCSF(4) # Set initial ion concentrations from Bevan & Wilson (1999)

    h.init()
    return {'electrodes': [stim1, stim2, stim3]}

# Gillies (2005)

## Make Cell

In [ ]:
gillies.load_template('STN_gillies_network')
icell = h.make_stn_cell_ref()

soma = icell.soma[0]
dendritic = list(icell.basal)

distal_seg = dendritic[8](0.75)
middle_seg = dendritic[20](0.25)
proximal_seg = dendritic[24](0.5)

In [ ]:
# TODO: look in calibrate_synapses_STN.ipynb, stn_model_evaluation.py, proto_gillies_article.py
rec_secs = {
    'soma': soma,
    'prox': proximal_seg,
    'mid': middle_seg,
    'dist': distal_seg,
}

trace_specs = collections.OrderedDict([
    ('V_soma', {'var':'v', 'sec':'soma', 'loc':0.5}),
    ('V_prox', {'var':'v', 'seg':'prox'}),
    ('V_mid', {'var':'v', 'seg':'mid'}),
    ('V_dist', {'var':'v', 'seg':'dist'}),
])

# Record
rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

## Cell Properties

In [ ]:
leaves = treeutils.leaf_sections(list(icell.basal)[0], subtree=True)
root = soma

In [ ]:
soma_Zin = {
    f : electrotonic.measure_input_impedance(soma(0.5), freq=f, linearize_gating=False)
                for f in (0.0, 100.0)
}

print("Somatic input impedances are: \n\n\t" + "\n\t".join(
        ["Zin(f={}) = {} (MOhm)".format(f, Z) for f,Z in soma_Zin.items()]))

measurements = plot_electrotonic_structure(root, leaves)

## Protocol SPONT

In [ ]:
# Gillies (2006) - Fig. 3.B (25 deg, 8Hz), 3.C (35 deg, 12 Hz)
setup_protocol_SPONT()
h.set_aCSF(4) # Set initial ion concentrations from Bevan & Wilson (1999)
h.run()

In [ ]:
figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True)

## Protocol REBOUND

In [ ]:
# Gillies (2006) - Fig. 3.D, 4 (altered conductances)
protocol = setup_protocol_REBOUND(icell)
h.set_aCSF(4) # Set initial ion concentrations from Bevan & Wilson (1999)
h.run()

In [ ]:
figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True)

## Protocol PLATEAU

In [ ]:
# Gillies (2006) - Fig. 10.C
# Protocol: hyperpolarization to -70 mV using constant current,
# then apply 0.2 nA, 50 ms depolarizing pulse, return to constant hyperpolarizing current
protocol = setup_protocol_PLATEAU(icell)
h.set_aCSF(4) # Set initial ion concentrations from Bevan & Wilson (1999)
h.run()

In [ ]:
figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True)

# Gillies from SWC

## Make Cell

In [ ]:
from bgcellmodels.models.STN.Miocinovic2006 import miocinovic_pynn_model as mio_pynn

morphology_path = os.path.join(miocinovic.pkg_dir, 'morphologies/gillies_original.swc')
streamlines_path = '/home/luye/Documents/mri_data/Waxholm_rat_brain_atlas/WHS_DTI_v1_ALS/S56280_ROI-STN-bilateral.tck'
template_name = 'STN_morph_arcdist'

model = mio_pynn.GilliesSwcModel(
            template_name='STN_morph_arcdist',
            morphology_path=morphology_path,
            streamlines_path=streamlines_path,
            axon_class=AxonMcintyre2002,
            without_extracellular=True)

icell = model.icell

In [ ]:
soma = icell.soma[0]
dendritic = list(icell.basal)
axonal = list(icell.axonal)

# indices for morphology 'type1RD_axonless-with-AIS'
distal_seg = icell.dend[7](0.8)
middle_seg = icell.dend[25](0.2)
proximal_seg = icell.dend[0](0.5)

axon_leaf_secs = treeutils.leaf_sections(list(icell.axonal)[0], subtree=True)
dend_leaf_secs = treeutils.leaf_sections(soma, subtree=True)
assert len(axon_leaf_secs) == 1, "More than one axon terminal section."

leaves = dend_leaf_secs
root = icell.soma[0]
axon_terminal = axon_leaf_secs[0]

rec_secs = {
    'soma': soma,
    'prox': proximal_seg,
    'mid': middle_seg,
    'dist': distal_seg,
    'ax_prox': model.axon['node'][0], # axonal[int(0.2 * len(axonal))],
    'ax_mid': model.axon['node'][1], # axonal[int(0.5 * len(axonal))],
    'ax_dist': model.axon['node'][2], # axonal[int(0.8 * len(axonal))],
    'ax_term': model.axon['node'][3], # axon_terminal,
}

# Miocinovic (2006)

## Make Cell

In [ ]:
#########################################################
# GENERIC TEMPLATE

# SETPARAM: morphology, axon/streamline, Hoc template
morphology_path = os.path.join(miocinovic.pkg_dir, 'morphologies/type1RD_axonless-with-AIS.swc')
streamlines_path = None # '/home/luye/Documents/mri_data/Waxholm_rat_brain_atlas/WHS_DTI_v1_ALS/S56280_track_filter-ROI-STN.tck'
template_name = 'STN_morph_arcdist' # default_template, 'STN_morph_cartdist', STN_morph_arcdist

model = template_with_morphology(
                template_name, morphology_path,
                streamlines_path, streamline_index=0)

icell = model.icell

# PREDEFINED TEMPLATE
# icell = template_predefined_morphology()

# Verify axon creation:
# print(list(icell.axonal))
# h.topology()

In [ ]:
soma = icell.soma[0]
dendritic = list(icell.basal)
axonal = list(icell.axonal)

# indices for morphology 'type1RD_axonless-with-AIS'
distal_seg = icell.dend[59](0.9) # 15,19,21,26,58,59,49,60,51
middle_seg = icell.dend[31](0.5) # 57, 31, 12
proximal_seg = icell.dend[8](0.2) # 8, 22, 29, 42

axon_leaf_secs = treeutils.leaf_sections(list(icell.axonal)[0], subtree=True)
dend_leaf_secs = treeutils.leaf_sections(soma, subtree=True)
assert len(axon_leaf_secs) == 1, "More than one axon terminal section."

leaves = dend_leaf_secs
root = icell.soma[0]
axon_terminal = axon_leaf_secs[0]

rec_secs = {
    'soma': soma,
    'prox': proximal_seg,
    'mid': middle_seg,
    'dist': distal_seg,
    'ax_prox': axonal[int(0.2 * len(axonal))],
    'ax_mid': axonal[int(0.5 * len(axonal))],
    'ax_dist': axonal[int(0.8 * len(axonal))],
    'ax_term': axon_terminal,
}

# Run Protocols

In [ ]:
trace_specs = collections.OrderedDict([
    ('V_soma', {'var':'v', 'sec':'soma', 'loc':0.5}),
    ('V_prox', {'var':'v', 'seg':'prox'}),
    ('V_mid', {'var':'v', 'seg':'mid'}),
    ('V_dist', {'var':'v', 'seg':'dist'}),
    ('V_axprox', {'var':'v', 'sec':'ax_prox', 'loc':0.5}),
    ('V_axmid', {'var':'v', 'sec':'ax_mid', 'loc':0.5}),
    ('V_axdist', {'var':'v', 'sec':'ax_dist', 'loc':0.5}),
    ('V_axterm', {'var':'v', 'sec':'ax_term', 'loc':0.5}),
])

# Record
rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

## Cell Properties

In [ ]:
# from neuron import gui
h.topology()

In [ ]:
# First draw skeleton to get an idea
# plt.figure()
# ax = plt.subplot(111, projection='3d')
# ax.view_init(90,90) # set azimut and elevation
# ax.set_zlim((-1,1)) # prevent rounding errors
# ax.set_axis_off() # hide x,y,z axes and grid

# # center = np.array([h.x3d(0, sec=soma), h.y3d(0, sec=soma), h.z3d(0, sec=soma)])
# x_soma = h.x3d(0, sec=soma)
# y_soma = h.y3d(0, sec=soma)
# ax.set_xlim((x_soma-200, x_soma+200))
# ax.set_ylim((y_soma-200, y_soma+200))


# # Plot shape of our cell
# morph_gui.shapeplot(h, ax)

# # Mark location of leaf section to refer to plots
# marker_colors = 'rgby'
# for i, leaf_sec in enumerate(leaves):
#     morph_gui.mark_locations(h, leaf_sec, 1.0, 
#                              label=leaf_sec.name(),
#                              markspec='o'+marker_colors[i])

# plt.legend()
# plt.show()

In [ ]:
soma_Zin = {
    f : electrotonic.measure_input_impedance(soma(0.5), freq=f, linearize_gating=False)
                for f in (0.0, 100.0)
}

print("Somatic input impedances are: \n\n\t" + "\n\t".join(
        ["Zin(f={}) = {} (MOhm)".format(f, Z) for f,Z in soma_Zin.items()]))

measurements = plot_electrotonic_structure(root, leaves)

## Protocol SPONT

In [ ]:
# Gillies (2006) - Fig. 3.B (25 deg, 8Hz), 3.C (35 deg, 12 Hz)
setup_protocol_SPONT()
h.run()

In [ ]:
figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True)

## Protocol REBOUND

In [ ]:
# Gillies (2006) - Fig. 3.D, 4 (altered conductances)
protocol = setup_protocol_REBOUND(icell)
h.run()

In [ ]:
figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True)

## Protocol PLATEAU

In [ ]:
# Gillies (2006) - Fig. 10.C
# Protocol: hyperpolarization to -70 mV using constant current,
# then apply 0.2 nA, 50 ms depolarizing pulse, return to constant hyperpolarizing current
protocol = setup_protocol_PLATEAU(icell)
h.run()

In [ ]:
figs_vm = analysis.plotTraces(vec_dict, rec_dt, traceSharex=True)

# Bevan, Chu (2016)

Can be found on neuromorpho.org using following searches
- [keyword 'subthalamic', sorty by archive](http://neuromorpho.org/KeywordBrowseView.jsp?count=52&keywords=%22subthalamic%22&browseBy=archive)
- [search by brain region 'basal ganglia'](http://neuromorpho.org/byregion.jsp#top)

# Save Notebook

In [ ]:
%%javascript
require(["base/js/namespace"],function(Jupyter) {
    Jupyter.notebook.save_checkpoint();
});

In [ ]:
from datetime import datetime
nb_filename = 'calibrate_morphologies_STN.ipynb'
report_filename = 'calibrate_morphologies_STN_{}_{}.html'.format(
    template_name, datetime.now().strftime('%Y.%m.%d_%H.%M.%S'))
!jupyter nbconvert $nb_filename --to html --template=toc2 --output $report_filename